In [ ]:
%matplotlib widget
from bmcs_shell.api import WBCell5Param, WBTessellation5PBeta, WBNumTessellation, WBNumTessellationInvest, WBTessellationBase, WBNumTessellationBase, WBCell5ParamBeta, WBTessellation4P, WBCell4Param
import numpy as np
import matplotlib.pyplot as plt

## Generate, extrude and subtract the geometry using gmsh

In [ ]:
params = [{'a': 50.0, 'b': 488.7096992126691, 'c': 219.35035456988965, 'gamma': 1.1474627907495765}, {'a': 75.0, 'b': 490.49365795443356, 'c': 201.23489828631506, 'gamma': 1.0544352782787323}, {'a': 100.0, 'b': 493.47789764898204, 'c': 188.08268555670807, 'gamma': 0.9231728219305352, 'n_phi_plus': 3}, {'a': 125.0, 'b': 498.98924772523196, 'c': 184.16057467325248, 'gamma': 0.7460842175215365, 'n_phi_plus': 3}]
wbt = WBTessellation4P(
                        **params[1],
                         n_x_plus=2,
    n_phi_plus=3,
                         wireframe_width=5,
#                          trim_half_cells_along_y=True,
#                          trim_half_cells_along_x=True,
#                          align_outer_nodes_along_x=True,
)
wbt.gamma = np.pi/2-1e-3
wbt.interact()

In [ ]:
wbt.plot_folding_pattern(trimmed=False)

In [ ]:
I_Fi = wbt.I_Fi
# I_Fi

In [ ]:
X_Ia = np.round(wbt.X_Ia, 0)
# X_Ia

### Basic mesh generation:

In [ ]:
# See https://gitlab.onelab.info/gmsh/gmsh/-/blob/master/tutorials/python/t11.py
# dimTag means tuple of (dimention, tag). Where tag is like an ID

import gmsh
import sys
import numpy as np
import bmcs_utils.api as bu

gmsh.initialize()
gmsh.model.add("t11")
mesh_size = 30

xpoints = np.array([gmsh.model.occ.addPoint(*X_a, mesh_size) for X_a in X_Ia])

for I_i in I_Fi:
    xpoints1 = xpoints[I_i]
    curves = [gmsh.model.occ.addLine(xpoints1[k], xpoints1[k + 1]) for k in range(len(xpoints1) - 1)] + [gmsh.model.occ.addLine(xpoints1[-1], xpoints1[0])]
    cl = gmsh.model.occ.addCurveLoop(curves)
    pl = gmsh.model.occ.addPlaneSurface([cl])

# Meshing ---------------------------------------------------- ------------------------------------------------

gmsh.model.occ.synchronize()

gmsh.model.mesh.generate(2)
# gmsh.model.mesh.recombine()
# gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)
# gmsh.model.mesh.refine()

# Launch the GUI to see the results:
if '-nopopup' not in sys.argv:
    gmsh.fltk.run()
elem_types, elem_tags, node_tags = gmsh.model.mesh.getElements()
print(gmsh.model.mesh.getElements())

gmsh.finalize()

### Flipping negative normals to unite all normals point (upwards)

In [ ]:
X_Fia = X_Ia[I_Fi]
To_flip_F = np.cross(X_Fia[:, 1, :] - X_Fia[:, 0, :], X_Fia[:, 2, :] - X_Fia[:, 0, :])[:, 2] < 0 # all items where z of normal is negative
To_flip_F

In [ ]:
I_Fi[To_flip_F] = np.flip(I_Fi[To_flip_F], axis=1)
I_Fi

In [ ]:
I_Li = np.vstack(((I_Fi[:, (0, 1)]), (I_Fi[:, (0, 2)]), (I_Fi[:, (1, 2)])))
I_Li = np.sort(I_Li, axis=1)
I_Li = np.unique(I_Li, axis=0)
I_Li

### Verify section

In [ ]:
import bmcs_utils.api as bu
import k3d

X_Lia = X_Ia[I_Li]
path_2a = X_Lia[0, ...]

path = path_2a
points = np.array([[2, 2, 0], [-2, 2, 0], [-2, -2, 0], [2, -2, 0]])

points = bu.Extruder.transform_first_contour(path, points, adapt_dimensions=True)

print('path=', path)
print('points=', points)

plot = k3d.plot()

mesh = k3d.lines(points.astype(np.float32), [[0, 1], [1, 2], [2, 3], [3, 0]], color=0xc73737, indices_type='segment', width=0.5)
plot += mesh

mesh = k3d.lines(path.astype(np.float32), [[0, 1]], color=0, width=0.5)
plot += mesh
plot

In [ ]:
from scipy.spatial.transform import Rotation

def get_rot_matrix_around_vector(v, deg_angle):
    deg_angle = np.deg2rad(deg_angle)
    
    c = np.cos(deg_angle)
    s = np.sin(deg_angle)
    v_norm = v / np.sqrt(sum(v * v))

    # See: Rotation matrix from axis and angle (https://en.wikipedia.org/wiki/Rotation_matrix)
    cross_product_matrix = np.cross(v_norm, np.identity(v_norm.shape[0]) * -1)
    return c * np.identity(3) + s * cross_product_matrix + (1 - c) * np.outer(v_norm, v_norm)

def rotate_points_around_vector(cs_points, vector, deg_angle, center_of_rotation=None):
    if center_of_rotation is None:
        cs_points_center = np.sum(cs_points, axis=0)/cs_points.shape[0]
    else:
        cs_points_center = center_of_rotation
    cs_points_at_origin = cs_points - cs_points_center

    rot_around_path = get_rot_matrix_around_vector(vector, deg_angle)
    r = Rotation.from_matrix(rot_around_path)
    cs_points_at_origin = r.apply(cs_points_at_origin)

    cs_points = cs_points_at_origin + cs_points_center
    return cs_points

In [ ]:
# Test rotation
cs_points = np.array([[1, 0, 0], [0, 2, 0], [-1, 0, 0]])
vec = np.array([[1, 1, 0], [2, 2, 0]])
cs_points = bu.Extruder.transform_first_contour(vec, cs_points, adapt_dimensions=True)
cs_points_rotated = rotate_points_around_vector(cs_points, vec[1, :] - vec[0, :], 90, center_of_rotation=(cs_points[0, :] + cs_points[2, :])/2)
cs_points_rotated

import k3d
plot = k3d.plot()

mesh = k3d.lines(cs_points.astype(np.float32), [[0, 1], [1, 2], [2, 0]], color=0x072aca, indices_type='segment', width=0.07)
plot += mesh

mesh = k3d.lines(cs_points_rotated.astype(np.float32), [[0, 1], [1, 2], [2, 0]], color=0xc73737, indices_type='segment', width=0.03)
plot += mesh

mesh = k3d.lines(vec, [[0, 1]], color=0, width=0.1)
plot += mesh
plot

In [ ]:
diff_vec = path_2a[1, :] - path_2a[0, :]
path_2a[0, :] = path_2a[0, :] - 0.001 * diff_vec
path_2a[1, :] = path_2a[1, :] + 0.001 * diff_vec

In [ ]:
def create_creases(wbt, X_Ia, occ, mesh_size, thickness, fold_thickness, valley=True):
    creases = []
    
    X_Lia = X_Ia[wbt.I_V_Li] if valley else X_Ia[wbt.I_M_Li]
    lines_num = X_Lia.shape[0]

    for l in range(lines_num):
        path_2a = X_Lia[l, ...]

        # Pipe cross-section
        if valley:
            cs_points = np.array([[thickness-fold_thickness, 0, 0], [0, thickness - fold_thickness, 0], [-(thickness - fold_thickness), 0, 0]])
        else:
            cs_points = np.array([[thickness-fold_thickness, thickness, 0], [0, fold_thickness, 0], [-(thickness - fold_thickness), thickness, 0]])
        cs_points = bu.Extruder.transform_first_contour(path_2a, cs_points, adapt_dimensions=True)

        # Rotate the triangle around its base if it points down 90, 180 (90+90), 270 (180+90)
        # (for a square or circle, this is not needed)
        center_of_rotation = (cs_points[0, :] + cs_points[2, :])/2
        for angle in range(3):
            if np.any(cs_points[:, 2] < -0.001):
                if valley:
                    cs_points = rotate_points_around_vector(cs_points, path_2a[1, :] - path_2a[0, :], 90, center_of_rotation)
                else:
                    cs_points[:, 2] = -cs_points[:, 2]
            else:
                break

        points = []
        for point in cs_points:
            points.append(occ.addPoint(*point, mesh_size))
        lines = [occ.addLine(points[k], points[k + 1]) for k in range(len(points) - 1)] + [occ.addLine(points[-1], points[0])]
        cl = occ.addCurveLoop(lines)
        pl = occ.addPlaneSurface([cl])

        # Pipe path (wire)
        points = []
        points.append(occ.addPoint(*path_2a[0, :], mesh_size))
        points.append(occ.addPoint(*path_2a[1, :], mesh_size))
        line = occ.addLine(points[0], points[1])
        wire = occ.addWire(curveTags = [line])
        creases.append(occ.addPipe(dimTags = [(2, pl)], wireTag=wire)[0])
    return creases

In [ ]:
# See https://gitlab.onelab.info/gmsh/gmsh/-/blob/master/tutorials/python/t11.py
# dimTag means tuple of (dimention, tag). Where tag is like an ID

import gmsh
import sys
import numpy as np
import bmcs_utils.api as bu
from scipy.spatial.transform import Rotation

# Each dimension Dim has Tags refering to objects starting from 1, 
#  (a point is dim=1, a line, surface or more is dim=2 and a volume is dim=3)

gmsh.initialize()
gmsh.model.add("t11")
mesh_size = 0

thickness = 2
fold_thickness = 0.6

X_Ia = wbt.X_Ia
I_Fi = wbt.I_Fi
width = np.max(X_Ia[I_Fi][:, :, 0]) - np.min(X_Ia[I_Fi][:, :, 0])
length = np.max(X_Ia[I_Fi][:, :, 1]) - np.min(X_Ia[I_Fi][:, :, 1])
model_max_dim = 200
pattern_max_dim = max(width, length)
scale_factor = model_max_dim/pattern_max_dim
X_Ia = np.copy(wbt.X_Ia) * scale_factor
X_Ia[:, 2] = 0
I_Fi = wbt.I_Fi

# Adding creases to subtract later: ---------------------------------------------------------------
# creases_valley = create_creases(wbt, X_Ia, gmsh.model.occ, mesh_size, thickness, fold_thickness, valley=True)
# gmsh.model.occ.remove(dimTags=gmsh.model.occ.getEntities(dim=2), recursive=True)
# gmsh.model.occ.fuse(creases_valley, creases_valley)

creases_mountain = create_creases(wbt, X_Ia, gmsh.model.occ, mesh_size, thickness, fold_thickness, valley=False)
gmsh.model.occ.remove(dimTags=gmsh.model.occ.getEntities(dim=2), recursive=True)
gmsh.model.occ.fuse(creases_mountain, creases_mountain)[0]

creases = gmsh.model.occ.getEntities(dim=3)

# print('creases_valley', creases_valley)
print('creases_mountain', creases_mountain)
print('creases', creases)
print('vols', gmsh.model.occ.getEntities(dim=3))

# Adding outer area of the pattern with extrusion: ------------------------------------------------

# xpoints = np.array([gmsh.model.occ.addPoint(*X_a, mesh_size) for X_a in X_Ia])
# wb_facets = []
# for I_i in I_Fi:
#     xpoints1 = xpoints[I_i]
#     curves = [gmsh.model.occ.addLine(xpoints1[k], xpoints1[k + 1]) for k in range(len(xpoints1) - 1)] + [gmsh.model.occ.addLine(xpoints1[-1], xpoints1[0])]

#     cl = gmsh.model.occ.addCurveLoop(curves)
#     pl = gmsh.model.occ.addPlaneSurface([cl])
#     wb_facets.append(pl)
    
# # #     To generate quadrangles instead of triangles, we can simply add
# #     gmsh.model.mesh.setRecombine(1, pl)

# # pg = gmsh.model.addPhysicalGroup(dim = 3, tags=wb_facets, name='pg')
# # print(pg)

# # Extrude (extrude is already a volume or CAD object)
# ext_facets = []
# for wb_facet in wb_facets:
#     ext_vol = gmsh.model.occ.extrude(dimTags=[(2, wb_facet)], dx=0, dy=0, dz=thickness, 
#                                  numElements=[], heights=[], recombine=True)[1]
#     ext_facets.append(ext_vol)
# ext_tessel = gmsh.model.occ.fuse(ext_facets, ext_facets)[0]
# print('ext_tessel', ext_tessel)

# # tess_block = [vol for vol in vols if vol not in creases]

# vols = gmsh.model.occ.getEntities(dim=3)
# print('vols', vols)

# # gmsh.model.occ.cut(ext_tessel, creases)
# gmsh.model.occ.remove(dimTags=gmsh.model.occ.getEntities(dim=2), recursive=True)

# print('cutted')
# # max3DTag = gmsh.model.occ.getMaxTag(3)

# Meshing ---------------------------------------------------- ------------------------------------------------

gmsh.model.occ.synchronize()

# field = gmsh.model.mesh.field
# field.add("MathEval", 1)
# field.setString(1, "F", "1")
# field.setAsBackgroundMesh(1)

# # To generate quadrangles instead of triangles, we can simply add
# gmsh.model.mesh.setRecombine(2, pl)

# If we'd had several surfaces, we could have used the global option
# "Mesh.RecombineAll":
#
# gmsh.option.setNumber("Mesh.RecombineAll", 1)

# You can also set the subdivision step alone, with
#
# gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)

# gmsh.model.mesh.generate(2)

# Note that you could also apply the recombination algorithm and/or the
# subdivision step explicitly after meshing, as follows:
#
print('before generate')
# gmsh.model.mesh.generate(2)
print('mesh generated')
# gmsh.model.mesh.recombine()
print('mesh recombine')
# gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)
# gmsh.model.mesh.refine()

# Launch the GUI to see the results:
if '-nopopup' not in sys.argv:
    gmsh.fltk.run()
    
# gmsh.write('test_wb.stl')
# gmsh.write('test_wb.step')

gmsh.finalize()

In [ ]:
# Volumes created from faces doesn't perform well for CAD boolean operations,
#  bettwer create CAD object, such as extrude or ready to use objects like box, sphere...
import gmsh
import math
import sys

gmsh.initialize()

gmsh.model.add("t16")

# Let's build the same model as in `t5.py', but using constructive solid
# geometry.

# We can log all messages for further processing with:
gmsh.logger.start()

# We first create two cubes:
# gmsh.model.occ.addBox(0, 0, 0, 1, 1, 1, 1)
# tunnels_vol = gmsh.model.occ.addBox(0.3, 0.3, 0.3, 0.5, 0.5, 0.5, 2)
# tunnels_vol = gmsh.model.occ.addSphere(1.3, 0, 0, 0.5)


points = []
points.append(gmsh.model.occ.addPoint(0, 0, 0))
points.append(gmsh.model.occ.addPoint(1, 0, 0))
points.append(gmsh.model.occ.addPoint(1, 1, 0))
points.append(gmsh.model.occ.addPoint(0, 1, 0))
lines = [gmsh.model.occ.addLine(points[k], points[k + 1]) for k in range(len(points) - 1)] + [gmsh.model.occ.addLine(points[-1], points[0])]
cl = gmsh.model.occ.addCurveLoop(lines)
pl = gmsh.model.occ.addPlaneSurface([cl])
gmsh.model.occ.extrude(dimTags=[(2, pl)], dx=0, dy=0, dz=2, numElements=[], heights=[], recombine=True)


points = []
points.append(gmsh.model.occ.addPoint(1, 0, 0))
points.append(gmsh.model.occ.addPoint(2, 0, 0))
points.append(gmsh.model.occ.addPoint(2, 1, 0))
points.append(gmsh.model.occ.addPoint(1, 0.5, 0))
lines = [gmsh.model.occ.addLine(points[k], points[k + 1]) for k in range(len(points) - 1)] + [gmsh.model.occ.addLine(points[-1], points[0])]
cl = gmsh.model.occ.addCurveLoop(lines)
pl = gmsh.model.occ.addPlaneSurface([cl])
gmsh.model.occ.extrude(dimTags=[(2, pl)], dx=0, dy=0, dz=2, numElements=[], heights=[], recombine=True)

points = []
points.append(gmsh.model.occ.addPoint(-0.2, 0.2, 0.2))
points.append(gmsh.model.occ.addPoint(0.5, 0.2, 0.2))
points.append(gmsh.model.occ.addPoint(0.5, 0.5, 0.2))
points.append(gmsh.model.occ.addPoint(-0.2, 0.5, 0.2))
lines = [gmsh.model.occ.addLine(points[k], points[k + 1]) for k in range(len(points) - 1)] + [gmsh.model.occ.addLine(points[-1], points[0])]
cl = gmsh.model.occ.addCurveLoop(lines)
pl = gmsh.model.occ.addPlaneSurface([cl])
gmsh.model.occ.extrude(dimTags=[(2, pl)], dx=0, dy=0, dz=0.5, numElements=[], heights=[], recombine=True)

# Pipe:
#-------
# Pipe cross-section
points = []
points.append(gmsh.model.occ.addPoint(0, 0, 0))
points.append(gmsh.model.occ.addPoint(0.1, 0, 0))
points.append(gmsh.model.occ.addPoint(0.1, 0.3, 0))
points.append(gmsh.model.occ.addPoint(0, 0.3, 0))
lines = [gmsh.model.occ.addLine(points[k], points[k + 1]) for k in range(len(points) - 1)] + [gmsh.model.occ.addLine(points[-1], points[0])]
cl = gmsh.model.occ.addCurveLoop(lines)
pl = gmsh.model.occ.addPlaneSurface([cl])

# Pipe path (wire)
points = []
points.append(gmsh.model.occ.addPoint(0, 0, 0))
points.append(gmsh.model.occ.addPoint(1, 1, 1))
line = gmsh.model.occ.addLine(points[0], points[1])
wire = gmsh.model.occ.addWire(curveTags = [line])
pipe = gmsh.model.occ.addPipe(dimTags = [(2, cl)], wireTag=wire)

# We apply a boolean difference to create the "cube minus one eigth" shape:
gmsh.model.occ.cut([(3, 1)], [(3, 3)])

# There's still following boolean:
# intersect (cut opposite) and fragment (union with keeping intersecting parts) and fuse (union)

gmsh.model.occ.synchronize()

gmsh.model.mesh.generate(3)

gmsh.write("t16.msh")

# Additional examples created with the OpenCASCADE geometry kernel are available
# in `t18.py', `t19.py' and `t20.py', as well as in the `examples/api'
# directory.

# Inspect the log:
log = gmsh.logger.get()
print("Logger has recorded " + str(len(log)) + " lines")
gmsh.logger.stop()

# Launch the GUI to see the results:
if '-nopopup' not in sys.argv:
    gmsh.fltk.run()

gmsh.finalize()


In [ ]:
wbt = WBTessellation5PBeta(wb_cell = WBCell5ParamBeta(a = 50, eta=1, zeta=1, gamma = np.pi/2-1e-3), n_x=2, n_y=2, wireframe_width=1)
wbt.interact()

In [ ]:
wb = WBCell4Param()
wb.interact()

In [ ]:
wb.X_Ia

In [ ]:
wb.I_Fi

In [ ]:
X_Ia[:, -1]+100

In [ ]:
vertices

In [ ]:
import k3d

thickness = 100
X_Ia = wb.X_Ia
I_Fi = wb.I_Fi

plot = k3d.plot()

mesh1 = k3d.mesh(X_Ia, I_Fi, side='double', color=0x999999)
plot+= mesh1
mesh1 = k3d.mesh(X_Ia, I_Fi, side='double', wireframe=True, color=0)
plot+= mesh1

# z_X_Ia = np.copy(X_Ia)
# z_X_Ia[:, -1] += thickness
# mesh2 = k3d.mesh(z_X_Ia, I_Fi, side='double', color=0x999999)
# plot+= mesh2

plot

In [ ]:
# Write to obj file
f = open('wb_3d_print.obj', 'w')
f.write('# Vertices: (' + str(len(vertices)) + ')\n')
for v in X_Ia:
    f.write('v ' + str(v)[1:-1] + '\n')
f.write('\n# Tri Facets: (' + str(len(indices)) + ')\n')
for ind in I_Fi + 1:
    f.write('f ' + str(ind)[1:-1] + '\n')
f.close()